In [ ]:
import numpy as np
import os
import PIL
import shutil
import pathlib
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array, load_img

In [ ]:
# Data preparation
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [ ]:
# Model construction
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Model training
history = model.fit(X_train, y_train, epochs=10, batch_size=128, validation_split=0.2)

In [ ]:
# Model eval
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)
print('Loss:', test_loss)

# Save model
model.save('digit_recognition_model.h5')

In [ ]:
!wget "https://www.dropbox.com/s/y9bs6r4nc7pj48c/digits.zip?dl=1" -O digits.zip
!unzip digits.zip
data_dir = "/content/digits"
output_dir = "/content/categorized_digits"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Image preprocessing
def load_and_preprocess_image(image_path):
    img = load_img(image_path, color_mode='grayscale', target_size=(28, 28))
    img_array = img_to_array(img)
    img_array = img_array.reshape(1, 28, 28, 1)
    img_array = img_array / 255.0
    return img_array

# Make predictions and count digit occurrences
digit_counts = {i: 0 for i in range(10)}

for filename in tqdm(os.listdir(data_dir)):
    if filename.endswith(".jpg"):
        img_path = os.path.join(data_dir, filename)
        img_array = load_and_preprocess_image(img_path)  
        prediction = model.predict(img_array)
        predicted_digit = np.argmax(prediction)
        digit_counts[predicted_digit] += 1

        # Create a digit-specific subdirectory
        digit_dir = os.path.join(output_dir, str(predicted_digit))
        os.makedirs(digit_dir, exist_ok=True)

        # Move the image to the corresponding subdirectory
        new_image_path = os.path.join(digit_dir, filename)
        shutil.move(img_path, new_image_path)

print("Digit counts:", digit_counts)